In [4]:
!rm *

rm: cannot remove 'sample_data': Is a directory


In [0]:
# from google.colab import files
# a = files.upload()

In [6]:
import pandas as pd
# import io
# risk_df = pd.read_csv(io.BytesIO(a['HWA-Brovsi-WSA-Claims-Validated.csv']), index_col='CLM_REF')
risk_df = pd.read_csv('https://raw.githubusercontent.com/IoT-master/FraudDetection/master/HWA-Brovsi-WSA-Claims-Validated.csv', index_col='CLM_REF')
risk_df = risk_df.dropna()
risk_df = risk_df.drop(['Index'], axis=1)
risk_df['ClaimTime'] = pd.to_datetime(risk_df['CLM_DT'], format='%d/%m/%y') - pd.to_datetime(risk_df['RCPT_DT'], format='%d/%m/%y')
risk_df['DistriTime'] = pd.to_datetime(risk_df['EA_DT'], format='%d/%m/%y') - pd.to_datetime(risk_df['RCPT_DT'], format='%d/%m/%y')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
import torch
from torch.utils import data
import pandas as pd

class DataSplitter(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, dataframe, features, labels):
        'Initialization'
        super(DataSplitter, self).__init__()
        self.dataframe = dataframe
        self.labels = labels
        self.list_IDs = features

    def __len__(self):
        'Denotes the total number of samples'
        return self.dataframe.shape[0]

    def __getitem__(self, index):
        'Generates one sample of data'
        # Load data and get label
        X = torch.FloatTensor(self.dataframe[self.list_IDs].to_numpy())[index]
        y = (torch.LongTensor(self.dataframe[self.labels].to_numpy())).squeeze(1)[index]

        return X, y

In [8]:
risk_df.head()

,EMP_ID,CLM_SYS,WSA_TYP,FLEXBEN_TYPE,CLM_DT,CLM_YR,CLM_AMT,RCPT_DT,RCPT_DAY,DAY_TAG,CLM.STAT,REIMB_DT,REIMB_YR,EA_DT,RISK,ClaimTime,DistriTime
CLM_REF,,,,,,,,,,,,,,,,,
EFSA0087162,11869,WSA,FLXI,5.0,4/11/16,2016,492.0,29/9/16,Thursday,WEEKDAY,3,3/1/17,2017,21/9/17,1,36 days,357 days
EFSA0087207,8965,WSA,FLXI,4.0,9/11/16,2016,471.0,30/9/16,Friday,WEEKDAY,3,3/1/17,2017,21/9/17,1,40 days,356 days
EFSA0087210,8998,WSA,FLXI,6.0,9/11/16,2016,496.0,30/9/16,Friday,WEEKDAY,3,7/1/17,2017,21/9/17,1,40 days,356 days
EFSA0087208,2098,WSA,FLXI,2.0,9/11/16,2016,464.0,3/10/16,Monday,WEEKDAY,3,5/1/17,2017,21/9/17,1,37 days,353 days
EFSA0087236,9331,WSA,FLXI,2.0,11/11/16,2016,454.0,6/10/16,Thursday,WEEKDAY,3,2/1/17,2017,21/9/17,1,36 days,350 days


In [0]:
risk_df['ClaimTime'] = risk_df['ClaimTime'].apply(lambda x: x.days)

In [0]:
risk_df['DistriTime'] = risk_df['DistriTime'].apply(lambda x: x.days)

In [0]:
risk_df = risk_df.drop(['CLM_SYS', 'CLM_DT', 'RCPT_DT', 'REIMB_DT', 'EA_DT'], axis=1)

In [0]:
risk_df = pd.get_dummies(risk_df)

In [13]:
risk_df

,EMP_ID,FLEXBEN_TYPE,CLM_YR,CLM_AMT,CLM.STAT,REIMB_YR,RISK,ClaimTime,DistriTime,WSA_TYP_FLXI,WSA_TYP_MIIS,RCPT_DAY_Friday,RCPT_DAY_Monday,RCPT_DAY_Saturday,RCPT_DAY_Sunday,RCPT_DAY_Thursday,RCPT_DAY_Tuesday,RCPT_DAY_Wednesday,DAY_TAG_HOLIDAY,DAY_TAG_WEEKDAY,DAY_TAG_WEEKEND
CLM_REF,,,,,,,,,,,,,,,,,,,,,
EFSA0087162,11869,5.0,2016,492.00,3,2017,1,36,357,1,0,0,0,0,0,1,0,0,0,1,0
EFSA0087207,8965,4.0,2016,471.00,3,2017,1,40,356,1,0,1,0,0,0,0,0,0,0,1,0
EFSA0087210,8998,6.0,2016,496.00,3,2017,1,40,356,1,0,1,0,0,0,0,0,0,0,1,0
EFSA0087208,2098,2.0,2016,464.00,3,2017,1,37,353,1,0,0,1,0,0,0,0,0,0,1,0
EFSA0087236,9331,2.0,2016,454.00,3,2017,1,36,350,1,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EFSA0118916,10471,4.0,2018,500.00,3,2018,1,21,61,1,0,0,0,1,0,0,0,0,0,0,1
EFSA0119321,10426,4.0,2018,347.00,3,2018,0,29,61,1,0,0,0,1,0,0,0,0,0,0,1
EFSA0119454,7814,7.0,2018,75.47,3,2018,1,31,61,1,0,0,0,1,0,0,0,0,0,0,1


In [14]:
test_limit = int(risk_df.shape[0]*.8)
risk_train = risk_df[:test_limit]
risk_test = risk_df[test_limit:]
risk_train.shape, risk_test.shape

((13816, 21), (3455, 21))

In [0]:
x_features = list(risk_df.columns)
x_features.remove('RISK')

In [0]:
ds = DataSplitter(risk_train, x_features, ['RISK'])

In [0]:
params = {'batch_size': 50,
          'shuffle': True,
          'num_workers': 2}

In [0]:
# training_generator = data.DataLoader(ds, **params)

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=20, out_features=10)
        self.fc2 = nn.Linear(in_features=10, out_features=5)
        self.output = nn.Linear(in_features=5, out_features=2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.output(x)
        return x

In [20]:
model = ANN()
model

ANN(
  (fc1): Linear(in_features=20, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=5, bias=True)
  (output): Linear(in_features=5, out_features=2, bias=True)
)

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [0]:
with torch.cuda.device(0):
  training_generator = data.DataLoader(ds, **params)

In [24]:
%%time
epochs = 100
loss_arr = []
with torch.cuda.device(0):
  for i in range(epochs):
      for x_train, y_train in training_generator:
          y_hat = model.forward(x_train)
          loss = criterion(y_hat, y_train)
          loss_arr.append(loss)

      if i % 10 == 0:
          print(f'Epoch: {i} Loss: {loss}')

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

Epoch: 0 Loss: 0.6403015851974487
Epoch: 10 Loss: 0.5570825338363647
Epoch: 20 Loss: 0.5939094424247742
Epoch: 30 Loss: 0.625524640083313
Epoch: 40 Loss: 0.43380290269851685
Epoch: 50 Loss: 0.5623385906219482
Epoch: 60 Loss: 0.6396323442459106
Epoch: 70 Loss: 0.2432735115289688
Epoch: 80 Loss: 0.48314395546913147
Epoch: 90 Loss: 0.5682942271232605
CPU times: user 40.1 s, sys: 13.1 s, total: 53.2 s
Wall time: 45min 23s


In [26]:
ds2 = DataSplitter(risk_test, x_features, ['RISK'])
X_test = ds2[:][0]
y_test = ds2[:][1]
X_test.shape, y_test.shape

(torch.Size([3455, 20]), torch.Size([3455]))

In [0]:
preds = []
with torch.no_grad():
    for val in X_test:
        y_hat = model.forward(val)
        preds.append(y_hat.argmax().item())

In [28]:
df = pd.DataFrame({'Y': y_test, 'YHat': preds})
df['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(df['Y'], df['YHat'])]
df

,Y,YHat,Correct
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
3450,1,0,0
3451,0,0,1
3452,1,0,0
3453,1,0,0


In [29]:
df['Correct'].sum() / len(df)

0.7638205499276411

In [40]:
# b = files.upload()
# risk_df = pd.read_csv(io.BytesIO(b['HWA-Brovsi-WSA-Claims.csv']), index_col='CLM_REF')
risk_df2 = pd.read_csv('https://raw.githubusercontent.com/IoT-master/FraudDetection/master/HWA-Brovsi-WSA-Claims.csv', index_col='CLM_REF')
risk_df2 = risk_df2.dropna()
risk_df2['ClaimTime'] = pd.to_datetime(risk_df2['CLM_DT'], format='%d/%m/%y') - pd.to_datetime(risk_df2['RCPT_DT'], format='%d/%m/%y')
risk_df2['DistriTime'] = pd.to_datetime(risk_df2['EA_DT'], format='%d/%m/%y') - pd.to_datetime(risk_df2['RCPT_DT'], format='%d/%m/%y')
risk_df2['ClaimTime'] = risk_df2['ClaimTime'].apply(lambda x: x.days)
risk_df2['DistriTime'] = risk_df2['DistriTime'].apply(lambda x: x.days)
risk_df2 = risk_df2.drop(['Index', 'CLM_SYS', 'CLM_DT', 'RCPT_DT', 'REIMB_DT', 'EA_DT'], axis=1)
risk_df2 = pd.get_dummies(risk_df2)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
risk_df2.head()

,EMP_ID,FLEXBEN_TYPE,CLM_YR,CLM_AMT,CLM.STAT,REIMB_YR,ClaimTime,DistriTime,WSA_TYP_FLXI,WSA_TYP_MIIS,RCPT_DAY_Friday,RCPT_DAY_Monday,RCPT_DAY_Saturday,RCPT_DAY_Sunday,RCPT_DAY_Thursday,RCPT_DAY_Tuesday,RCPT_DAY_Wednesday,DAY_TAG_HOLIDAY,DAY_TAG_WEEKDAY,DAY_TAG_WEEKEND
CLM_REF,,,,,,,,,,,,,,,,,,,,
EFSA0087162,11869,5.0,2016,492.0,3,2017,36,357,1,0,0,0,0,0,1,0,0,0,1,0
EFSA0087207,8965,4.0,2016,471.0,3,2017,40,356,1,0,1,0,0,0,0,0,0,0,1,0
EFSA0087210,8998,6.0,2016,496.0,3,2017,40,356,1,0,1,0,0,0,0,0,0,0,1,0
EFSA0087208,2098,2.0,2016,464.0,3,2017,37,353,1,0,0,1,0,0,0,0,0,0,1,0
EFSA0087236,9331,2.0,2016,454.0,3,2017,36,350,1,0,0,0,0,0,1,0,0,0,1,0


In [0]:
x_val = torch.FloatTensor(risk_df2.to_numpy())

In [0]:
preds2 = []
with torch.no_grad():
    for val in x_val:
        y_hat = model.forward(val)
        preds2.append(y_hat.argmax().item())

In [0]:
df = pd.DataFrame(preds2) 
df.to_csv('predictions.csv') 

In [46]:
for each_matrix in model.parameters():
  print(each_matrix)

Parameter containing:
tensor([[-2.0640e-01, -2.5840e-02, -1.6329e-01,  9.2314e-02, -1.3945e-01,
         -4.1647e-02,  3.2516e-02,  1.1045e-01, -1.3764e-01,  1.2492e-01,
         -1.1641e-01, -2.3108e-03, -1.1723e-01,  1.4494e-01, -7.3914e-02,
         -2.0082e-01,  7.5567e-02,  9.0076e-02, -3.7604e-02, -3.7278e-02],
        [ 7.7819e-03, -2.4571e-01, -2.4010e-01, -1.3134e-01, -2.4660e-01,
         -1.6567e-01, -1.6203e-01, -2.2166e-01,  5.4706e-02, -9.8691e-02,
          1.1383e-02,  8.5480e-02, -2.0716e-01,  5.0834e-02, -1.2506e-01,
         -1.7294e-01,  2.9548e-02, -9.2822e-02, -2.4140e-01,  8.9915e-02],
        [ 4.9448e-02, -1.5781e-01, -2.2444e-01, -1.6426e-01, -1.0753e-01,
         -1.8584e-01, -5.3418e-02, -2.1863e-01, -5.6292e-02,  8.3317e-02,
         -1.4149e-01, -3.1789e-01, -9.7351e-03, -5.2981e-06,  2.3019e-01,
          4.9712e-02, -2.7051e-01, -2.7131e-01, -6.8496e-02, -1.8449e-01],
        [-2.1574e-01, -2.0167e-01,  6.8130e-02, -1.7273e-01, -1.9609e-01,
          2.0

In [0]:
from google.colab import files
files.download('predictions.csv') 